In [1]:
#>cd Documents/NTU/2_Sophomore/2-2/R/Project/LING5505-Final-Project-Group3

In [1]:
import numpy as np
import pandas as pd
import gdown
import os

import torch
import transformers

import CwnSenseTagger
CwnSenseTagger.download()

import CwnGraph
CwnGraph.download()

from CwnGraph import CwnBase
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

#import DistilTag
#DistilTag.download()

A copy of DistilTag model already exists. Use upgrade=True to overwrite
A copy of cwn_grpah.pyobj already exists. Use upgrade=True to overwrite


In [2]:
f = open("data_set/AST_100_1.txt")
AST_100_1 = f.read()

In [3]:
ws = WS(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)) + '/data')
pos = POS(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)) + '/data')
ner = NER(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)) + '/data')

/Users/dereklu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/dereklu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [4]:
sentence_list = [AST_100_1]

word_sentence_list = ws(sentence_list)

pos_sentence_list = pos(word_sentence_list)

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

### 斷詞結果顯示

In [5]:
def print_word_pos_sentence (word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)


'「寬」，是人生的廣度和豐富度；「深」，則是人生的厚度，寬與深看似人生的不同面向，但事實上，人生在寬與深的調和中，方能展現最強的生命韌度，擴展人生的價值。
古典物理學大師──牛頓使世人見識到了知識的深度、人生的寬度。年輕時即以高度的科學天分嶄露他的不凡，後來更發表了「萬有引力」和「牛頓三大運動定律」使他獲皇家科學院的殊榮，但他在深入探討力學後，更進一步轉往光學方面的研究，甚至老年時還竭盡心力研究微積分理論。他的成就不僅深遠而且層面寬廣，關鍵就在於他行事的認真、踏實和專注。在人生各階段中，都有「階段性任務」，若專心一意地將心力投注於各階段上，那人生的「寬與深」便能兼顧。
當然，欲發展人生的「寬」或「深」取決於自身能力和外在因素，並非都能像牛頓一般達到人生境界的最佳平衡點。史懷哲三十歲時便獲得了三種領域的博士學位，但在傾聽「內心的鼓聲」、理想的呼喚後，他決心習醫，並前往非洲落後地區行醫、助人。因為認清所求，他勇敢堅毅地在各領域中轉換，雖然因生命有限，而無法更深入地鑽研各領域的知識，但是，所學之廣豐富了他人文、科學各方面的才能，並且在實踐行醫理想的歲月中，他不也為自己的人生注入了更深的愛和關懷嗎？這，亦是另一層面的「深度」。
人生，是追尋和實踐的過程。在追尋中，我們將遭遇各式的選擇，體驗各色的人生風貌──這便是人生的「寬度」；在實踐中，我們竭盡心力灌溉夢想、追求卓越，在一次又一次的試煉與考驗中錘煉自我，增強能力──這便是人生的「深度」。牛頓和史懷哲皆依循自己的內心，勇敢畫出屬於自己獨特的「生命輿圖」，深與廣不再侷限他們的人生意義，因為他們已用堅定的信念和努力地付出，使「深與廣的調合」綻放獨特的生命價值。'
「(PARENTHESISCATEGORY)　寬(VH)　」(PARENTHESISCATEGORY)　，(COMMACATEGORY)　是(SHI)　人生(Na)　的(DE)　廣度(Na)　和(Caa)　豐富度(Na)　；(SEMICOLONCATEGORY)　「(PARENTHESISCATEGORY)　深(VH)　」(PARENTHESISCATEGORY)　，(COMMACATEGORY)　則(D)　是(SHI)　人生(Na)　的(DE)　厚度(Na)　，(COMMACATEGORY)　寬(VH)　與(Caa)　深(VH)　看似(VG)　人生(Na)　的(DE

In [6]:
pos_sentence_arr = pd.Series(np.concatenate(pos_sentence_list))
display(pos_sentence_arr.head())

0    PARENTHESISCATEGORY
1                     VH
2    PARENTHESISCATEGORY
3          COMMACATEGORY
4                    SHI
dtype: object

### 詞性頻率分析

In [7]:
pos_df = pos_sentence_arr.value_counts().sort_index().rename_axis('CKIP_POS').reset_index(name = 'frequency')
pos_df_sorted = pos_df.sort_values(by = ['frequency'], ascending = False)
pos_df_sorted.head()

,CKIP_POS,frequency
9,Na,94
6,DE,43
35,VH,40
1,COMMACATEGORY,38
4,D,30


In [8]:
word_sentence_arr = pd.Series(np.concatenate(word_sentence_list))
word_sentence_arr

0       「
1       寬
2       」
3       ，
4       是
       ..
482    獨特
483     的
484    生命
485    價值
486     。
Length: 487, dtype: object

### 辭意搜尋與辭意計算

In [9]:
cwn = CwnBase()
    
def all_sense_tree (word, verbal = False):
    cnt = 0
    for i in range(len(word)):
        snese_tree = word[i].senses
        cnt += len(word[i].senses)
        if(verbal == True): print(snese_tree)
    
    if(verbal == True): print("total senses = ", cnt)
    return cnt

In [10]:
_word = word_sentence_arr[484]
word = cwn.find_lemma("^" + _word + "$")
print("word: ", _word)
all_sense_tree(word, verbal = True)

word:  生命
[<CwnSense[06531701](生命): 生物生存的機能。>, <CwnSense[06531702](生命): 自然界中所有具有生命並有生長繁殖能力的物體。>, <CwnSense[06531703](生命): 比喻人的生活和經歷。>, <CwnSense[06531704](生命): 比喻前述對象重視、不可或缺的特定對象。>, <CwnSense[06531705](生命): 比喻人造物發揮功能的時段。>, <CwnSense[06531706](生命): 比喻生物從事前述事件的一段生命時間範圍。>, <CwnSense[06531707](生命): 比喻藝術作品靈活生動、貼近人類生活的特質。>, <CwnSense[06531708](生命): 比喻可使特定對象正面發展的機會。>]
total senses =  8


8

### 所有單詞詞性與辭意計算（辭意非零者）

In [11]:
pun_set = {"COLONCATEGORY", "COMMACATEGORY", "DASHCATEGORY", "ETCCATEGORY", "EXCLAMATIONCATEGORY", "PARENTHESISCATEGORY",
          "PAUSECATEGORY", "PERIODCATEGORY", "QUESTIONCATEGORY", "SEMICOLONCATEGORY", "SPCHANGECATEGORY"}

senses_list = list()

for i in range(len(word_sentence_arr)):
    if(pos_sentence_arr[i] not in pun_set):
        _word = word_sentence_arr[i]
        word = cwn.find_lemma("^" + _word + "$")
        sense_cnt = all_sense_tree(word)
        senses_list.append((word_sentence_arr[i], pos_sentence_arr[i], sense_cnt))
        
senses_df = pd.DataFrame(senses_list, columns = ['Word', 'CKIP_POS', 'sense_cnt'])
senses_df_ = senses_df[senses_df.sense_cnt != 0]
senses_df_

,Word,CKIP_POS,sense_cnt
0,寬,VH,8
1,是,SHI,9
2,人生,Na,1
3,的,DE,16
5,和,Caa,16
...,...,...,...
393,的,DE,16
396,獨特,VH,1
397,的,DE,16
398,生命,Na,8


### 辭意為零者

In [12]:
senses_df_0 = senses_df[senses_df.sense_cnt == 0]
senses_df_0

,Word,CKIP_POS,sense_cnt
4,廣度,Na,0
6,豐富度,Na,0
12,厚度,Na,0
20,面向,Na,0
30,調和,VH,0
...,...,...,...
368,輿圖,Na,0
382,堅定,VHC,0
384,信念,Na,0
394,調合,VC,0


### 辭意非零者平均辭意

In [13]:
# count average senses
senses_df_['sense_cnt'].mean()

7.901898734177215

### 扣除詞性為DE與Nh者單詞出現頻率

In [14]:
senses_df_adj = senses_df_[senses_df_.CKIP_POS != "DE"]
senses_df_adj = senses_df_adj[senses_df_adj.CKIP_POS != "Nh"]

words_cnt_adj = pd.DataFrame(senses_df_adj.value_counts())
words_cnt_adj

,,,0
Word,CKIP_POS,sense_cnt,
人生,Na,1,16
在,P,10,9
和,Caa,16,8
深,VH,17,7
各,Nes,2,7
...,...,...,...
大,VH,28,1
地區,Nc,1,1
在於,VK,1,1
